In [1]:
import cv2
import time
import os
import dlib
import numpy as np


In [2]:
#Definimos el método que detectará los rostros
def detectFaceDlibHog(detector, frame, inHeight=300, inWidth=0):

    frameDlibHog = frame.copy()
    frameHeight = frameDlibHog.shape[0]
    frameWidth = frameDlibHog.shape[1]
    if not inWidth:
        inWidth = int((frameWidth / frameHeight)*inHeight)

    scaleHeight = frameHeight / inHeight
    scaleWidth = frameWidth / inWidth

    frameDlibHogSmall = cv2.resize(frameDlibHog, (inWidth, inHeight))

    frameDlibHogSmall = cv2.cvtColor(frameDlibHogSmall, cv2.COLOR_BGR2RGB)
    faceRects = detector(frameDlibHogSmall, 0)
   
    bboxes = []
    for faceRect in faceRects:
        
        cvRect = [int(faceRect.left()*scaleWidth), int(faceRect.top()*scaleHeight),
                  int(faceRect.right()*scaleWidth), int(faceRect.bottom()*scaleHeight) ]
        bboxes.append(cvRect)
        cv2.rectangle(frameDlibHog, (cvRect[0], cvRect[1]), (cvRect[2], cvRect[3]), (0, 255, 0), int(round(frameHeight/150)), 4)
    return frameDlibHog, bboxes


In [4]:
#Cargamos nuesto clasificador
hogFaceDetector = dlib.get_frontal_face_detector()

rutaDatasets = '../../Videos/Datasets/'
nombreDataset = 'CamNet 352 x 288/'
#Asigamos un directorio donde se guardarán nuestros archivos
directorioSalida ='../../Videos/Resultados/Salida Hog Rostros/'

try:
    # Creamos el directorio de salida
    os.mkdir(directorioSalida)
    print( directorioSalida ,  " Creado ") 
except FileExistsError:
    # Si ya esxistía, se cacha la excepción
    print( directorioSalida ,  " Ya existia")

try:
    #Creamos los de salida
    f = open(directorioSalida+"Salida HAAR.csv", "x")
    f2 = open(directorioSalida+"Conteo de activacion HOG Rostros.csv", "x")
except:
    #Si ya existiían, se procede a abrilos
    f = open(directorioSalida+"Salida HAAR.csv", "w")
    f2 = open(directorioSalida+"Conteo de HOG Rostros.csv", "w")

contador = 1
#Se colocán las cabeceras de los archivos
f.write("nombre, tiempo, fps \n")
f2.write("nombre, cuadros Totales, cuadrosActivados \n")

#En este ciclo 'for' se toman en cuenta todos los vídeos que se quieran procesar
for i in range(8): 
    #Esta bandera es para indicar que aún nos encontramos trábjando con un vídeo en espeficico
    ban = True
    #El contador de cuadros se inicializa en 0 para cada vídeo
    contadorCuadros = 0
    #Se da el nombre de entrada del video, en este caso los videos 
    #son nombrados con "video_n" para automatizar el proceso
    nombreEntrada = "video_"+str(i+1) 

    #Se carga el vídeo que queremos utilizar
    cap=cv2.VideoCapture(rutaDatasets+nombreDataset+nombreEntrada+'.mp4')
    #Obtenemos los FPS
    fps = cap.get(cv2.CAP_PROP_FPS)

    #Definimos tamaño a redimensionar
    frame_width = 352#
    frame_height = 288#

    #Le damos un nombre al vídeo que tendremos como salida
    nombreSalida = "salida"+str(contador)+"_"+str(fps)+"fps"

    # Definimos el codec y guardamos el video
    out = cv2.VideoWriter(directorioSalida+nombreSalida+'.avi',cv2.VideoWriter_fourcc('M','J','P','G'), int(fps), (frame_width,frame_height))
    #Asignamos el contador de activación en 0
    contadorActivacion = 0 
    #Asignamos un tiempo de inicio
    inicio = time.time()
    #Este cliclo 'While' nos dice si los cuadros de un video fueron ya procesados
    while ban:
        #Se lee un cuadro del video
        r, frame = cap.read()
        #Se aumenta el contador de cuadros
        contadorCuadros = contadorCuadros + 1
        #se verifica si el vídeo aún tiene cuadros por procesar
        if r == True:
            #Este proceso es el mismo que se vío anteriormente
            frame = cv2.resize(frame, (frame_width, frame_height))

            found, bboxes = detectFaceDlibHog(hogFaceDetector,frame)

            try:
                if ( all(found)):
                    x = 1

            except:
                #En caso de que haya un fallo en found, se entiende que se activó el algoritmo
                #y aunmentamos el contador en uno 
                contadorActivacion = contadorActivacion + 1

            #Mostramos el found
            cv2.imshow('HOG Rostros',found)
            #Guardamos ese frame
            out.write(frame)

            key = cv2.waitKey(1)
            if key & 0xFF == ord('q'):
                ban = False
        else:
            #En caso de que no tenga cuadros por procesar se sale del ciclo
            ban = False

    contador = contador + 1
    fin = time.time()
    segundos = fin - inicio
    fpsNuevos = contadorCuadros/segundos
    #Se guardan los resultados en los archivos
    f.write(str(nombreEntrada) + ',' + str(segundos) + ',' + str(fpsNuevos)+ '\n')
    f2.write(str(nombreEntrada) + ',' + str(contadorCuadros) + ',' + str(contadorActivacion)+'\n')
    #Se imprimen por pantalla los resulatodos
    print(nombreEntrada + " FPS: " +str(fpsNuevos)) 
    print(nombreEntrada + " Tiempo total: " + str(segundos))
    #Se cierran los capturadores
    cap.release()
    out.release()

#Se cierran los archivos
f.close()    
f2.close()
cv2.destroyAllWindows()

../../Videos/Resultados/Salida Hog Rostros/  Creado 
video_1 FPS: 31.469717690696392
video_1 Tiempo total: 2.0972542762756348
video_2 FPS: 32.25085254032025
video_2 Tiempo total: 0.18604159355163574
video_3 FPS: 35.49476516958368
video_3 Tiempo total: 0.16903901100158691
video_4 FPS: 32.04425055962595
video_4 Tiempo total: 0.15603423118591309
video_5 FPS: 32.07839798090528
video_5 Tiempo total: 0.1870417594909668
video_6 FPS: 35.70663768420276
video_6 Tiempo total: 0.16803598403930664
video_7 FPS: 32.460236383786224
video_7 Tiempo total: 0.15403461456298828
video_8 FPS: 35.70628305010918
video_8 Tiempo total: 0.16803765296936035
